In [1]:
import xarray as xr
import glob
import geopandas as gpd
import sys
from   easymore import Easymore

In [2]:
path_org = '/home/shg096/scratch/Bow_at_Calgary/'

riv = gpd.read_file(path_org+'domain/Bow_at_Calgary_riv.shp')
riv.sort_values(by='COMID').reset_index(drop=True)
riv['lengthm'] = 0.00
riv['lengthm'] = riv['lengthkm'] * 1000 # km to m
riv['uparea'] = riv['uparea'] * 1000000 # km2 to m2

cat = gpd.read_file(path_org+'domain/Bow_at_Calgary_cat.shp')
cat.sort_values(by='COMID').reset_index(drop=True)
cat['area'] = 0.00
cat['area'] = cat['unitarea'] * 1000000 # km2 to m2
cat['latitude'] = cat.centroid.y
cat['longitude'] = cat.centroid.x


/tmp/ipykernel_22995/121174941.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cat['latitude'] = cat.centroid.y
/tmp/ipykernel_22995/121174941.py:14: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cat['longitude'] = cat.centroid.x


In [3]:
# check if the COMID are identical
if sum(riv['COMID'].values - cat['COMID'].values) != 0:
    sys.exit('The COMID of the shapefile in soil and land cover is not identical')

### pass the area from the cat to riv and also calculate the width based on Booker, 2010, Mizukami et al., 2016

In [4]:
riv['area'] = 0
riv['area'] = cat ['area']
riv['width'] = 0.001 * (riv['uparea']) ** 0.5

ds = riv.to_xarray()

ds

<xarray.Dataset>
Dimensions:     (index: 453)
Coordinates:
  * index       (index) int64 0 1 2 3 4 5 6 7 ... 446 447 448 449 450 451 452
Data variables: (12/19)
    COMID       (index) int64 71022153 71022160 71022164 ... 71039250 71039266
    lengthkm    (index) float64 9.651 30.02 4.118 1.383 ... 5.537 2.518 11.98
    lengthdir   (index) float64 6.073 23.71 3.242 1.22 ... 4.428 2.041 10.24
    sinuosity   (index) float64 1.589 1.266 1.27 1.134 ... 1.277 1.25 1.234 1.17
    slope       (index) float64 0.0009722 0.0008482 ... 0.03278 0.005844
    uparea      (index) float64 1.172e+10 1.142e+10 ... 2.796e+07 1.137e+08
    ...          ...
    up3         (index) int64 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
    up4         (index) int64 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
    geometry    (index) object LINESTRING (-113.805 52.2775, -113.80583333333...
    lengthm     (index) float64 9.651e+03 3.002e+04 ... 2.518e+03 1.198e+04
    area        (index) float64 6.964e+07 1.084e+08 ... 9.346e+07 1.531e+08
    width       (index) float64 108.3 106.9 105.6 105.5 ... 7.218 5.288 10.66